In [21]:
from dataclasses import dataclass, field
from datetime import datetime
from time import time
from functools import total_ordering
import os
from sortedcontainers import SortedList

In [1]:
class OrderIdGenerator:
    def __init__(self):
        self.filepath = os.path.join(os.getcwd(), "current_state.txt")
        self.id_counter = self.load_state()
        self.iterations_since_last_save = 0
        self.save_frequency = 100

    def __iter__(self):
        return self

    def __next__(self):
        current_id = self.id_counter
        self.id_counter += 1
        self.iterations_since_last_save += 1
        if self.iterations_since_last_save >= self.save_frequency:
            self.save_state()
            self.iterations_since_last_save = 0

        return current_id

    def save_state(self):
        with open(self.filepath, "w") as f:
            f.write(str(self.id_counter))

    def load_state(self):
        if not os.path.exists(self.filepath):
            return 0
        with open(self.filepath, "r") as f:
            saved_state = f.read()
            return int(saved_state)

    def reset_state(self):
        self.id_counter = 0
        self.save_state()


g_obj = OrderIdGenerator()
next(g_obj)

0

In [8]:
from enum import Enum, auto


class OrderStatus(Enum):
    CREATED = auto()
    PARTIALLY_FILLED = auto()
    FILLED = auto()
    CANCELLED = auto()
    

In [20]:
@dataclass
class Order:
    order_type: str = field(compare=False)
    security_name: str = field(compare=False)
    price: float = field(compare=True)
    volume: int = field(compare=False)
    owner_id: int = field(compare=False)
    status: int = field(compare=False, default=OrderStatus.CREATED)
    id: int = field(compare=False, default_factory=g_obj.__next__)
    time_created: float = field(compare=False, default_factory=time)
    last_update: float = field(compare=False, default_factory=time)

    def __eq__(self, other):
        if isinstance(other, (int, float)):
            return self.price == other
        if isinstance(other, Order):
            return self.price == other.price
        return NotImplemented

    def __lt__(self, other):
        if isinstance(other, (int, float)):
            return self.price < other
        if isinstance(other, Order):
            return self.price < other.price
        return NotImplemented

    def __gt__(self, other):
        if isinstance(other, (int, float)):
            return self.price > other
        if isinstance(other, Order):
            return self.price > other.price
        return NotImplemented

    def __le__(self, other):
        if isinstance(other, (int, float)):
            return self.price <= other
        if isinstance(other, Order):
            return self.price <= other.price
        return NotImplemented

    def __ge__(self, other):
        if isinstance(other, (int, float)):
            return self.price >= other
        if isinstance(other, Order):
            return self.price >= other.price
        return NotImplemented

    def __repr__(self):
        return (
            f"Order № {self.id} \n"
            f"Order Type {self.order_type} \n"
            f"Security: {self.security_name} \n"
            f"Price: {self.price} \n"
            f"Volume: {self.volume} \n"
            f"Owner: {self.owner_id} \n"
            f"Status: {self.status} \n"
            f"Time: {self.timestamp} \n"
        )

    def __post_init__(self):
        if self.volume <= 0:
            raise ValueError("Volume must be positive")
        if self.price <= 0:
            raise ValueError("Price must be positive")
        if self.order_type not in ["ask", "bid"]:
            raise ValueError("Order type must be 'ask' or 'bid'")
        if self.status not in OrderStatus:
            raise ValueError("Invalid order status")
    

In [ ]:
class OrderBook:
    
    
    def __init__(self):
        self.ask = SortedList()
        self.bid = SortedList()
        self.order_map = {}
        
    def add_order(self, order):
        if order.order_type == "ask":
            self.ask.add(order)
        else:
            self.bid.add(order)
        self.order_map[order.id] = order

    def cancel_order(self, order_id):
        for order in self.ask:
            if order.id == order_id:
                order.status = OrderStatus.CANCELLED
                self.ask.remove(order)
                return
        for order in self.bid:
            if order.id == order_id:
                order.status = OrderStatus.CANCELLED
                self.bid.remove(order)
                return
        raise ValueError("Order with such id not found")
    
    def update_order(self, order_id, new_volume):
        for order in self.ask:
            if order.id == order_id:
                order.volume = new_volume
                return
        for order in self.bid:
            if order.id == order_id:
                order.volume = new_volume
                return
        raise ValueError("Order with such id not found")
    

In [ ]:
from enum import Enum

class MarketSecurities(Enum):
    APPLE = 1
    GOOGLE = 2
    MICROSOFT = 3
    AMAZON = 4

# Example usage
security_id = MarketSecurities.APPLE.value
print(security_id)  # Output: 1


In [ ]:
from enum import Enum, auto

id_dict = {"AAPLE": 1, "GOOGLE": 2, "MICROSOFT": 3, "AMAZON": 4}

class MarketSecurities(Enum):
    pass

for name, value in id_dict.items():
    setattr(MarketSecurities, name, value)

# Example usage
security_id = MarketSecurities.APPLE.value
print(security_id)  # Output: 1


In [19]:
a = Order("bid", "Microsoft", 100, 10, 1)
print(a.fill(15))
a.status
# b = Order("bid", "Microsoft", 110, 10, 1)
# c = Order("bid", "Microsoft", 95, 10, 1)
# print(a)
# print(b)
# print(c)

30


<OrderStatus.PARTIALLY_FILLED: 2>

<OrderStatus.CANCELLED: 4>

In [11]:
from bintrees import AVLTree, RBTree

In [15]:
from sortedcontainers import SortedList

In [40]:
sl = SortedList([a, b, c])

In [42]:
d= Order("bid", "Microsoft", 80, 100, 1)

In [43]:
print(sl)
sl.add(d)
sl

SortedList([Order № 16 
Order Type bid 
Security: Microsoft 
Price: 95 
Volume: 10 
Owner: 1 
Status: 0 
Time: 1706132732.8304594 
, Order № 14 
Order Type bid 
Security: Microsoft 
Price: 100 
Volume: 10 
Owner: 1 
Status: 0 
Time: 1706132732.8304594 
, Order № 15 
Order Type bid 
Security: Microsoft 
Price: 110 
Volume: 10 
Owner: 1 
Status: 0 
Time: 1706132732.8304594 
])


SortedList([Order № 17 
Order Type bid 
Security: Microsoft 
Price: 80 
Volume: 100 
Owner: 1 
Status: 0 
Time: 1706133128.3177133 
, Order № 16 
Order Type bid 
Security: Microsoft 
Price: 95 
Volume: 10 
Owner: 1 
Status: 0 
Time: 1706132732.8304594 
, Order № 14 
Order Type bid 
Security: Microsoft 
Price: 100 
Volume: 10 
Owner: 1 
Status: 0 
Time: 1706132732.8304594 
, Order № 15 
Order Type bid 
Security: Microsoft 
Price: 110 
Volume: 10 
Owner: 1 
Status: 0 
Time: 1706132732.8304594 
])

In [ ]:
class OrderTree(SortedList):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._max_id = 0

    def insert(self, order):
        super().insert(order.id, order)
        self._max_id = max(self._max_id, order.id)

    def remove(self, order):
        super().remove(order.id)
        if order.id == self._max_id:
            self._max_id = max(self.keys())

    def get_max_id(self):
        return self._max_id